In [5]:
import os

In [6]:
import nltk

In [7]:
import pandas as pd

In [8]:
from sklearn.metrics import confusion_matrix

In [9]:
from TextNormalizer import *

In [10]:
import math

In [11]:
import time

In [12]:
class Model():
    def __init__(self):
        self.classes = {}
        self.logprob={} 
        self.vector_of_words=[]
        self.loglikelihood={}
               
    def trainNBClassifier(self):
         
        self.getTrainingData()    
        bigdoc={}
        
        for classe in self.classes:
            text=''
            
            number_of_documents = sum(map(lambda classes:int(classes),self.classes.values())) 
            number_of_doc_in_class = self.classes[classe]
            
            self.logprob[classe] = math.log(number_of_doc_in_class/number_of_documents)
        
            self.vector_of_words = self.extractVocabulary()
            
            files=os.listdir('./dataset/train/'+classe)
            for doc in files:
                text += self.read_file("./dataset/train/"+classe+"/"+doc)
            
            bigdoc[classe]= text
            
            words_of_classe = nltk.tokenize.word_tokenize(bigdoc[classe])
            
            for word in self.vector_of_words:
                self.loglikelihood[word,classe] = (bigdoc[classe].count(word) +1 / (len(words_of_classe)+len(self.vector_of_words)))
        
        print(self.classes)
        print(self.logprob)
        print(self.vector_of_words)
        print(self.loglikelihood)
                
        return (self.classes, self.logprob, self.vector_of_words, self.loglikelihood )
    
    def predict(self, file):
            
        words=[]
        probabilityPerClass={}
        doc= self.read_file(file)
        normalizer = TextNormalizer()
        text = normalizer.normalizing(doc)
        sentences = nltk.sent_tokenize(text)        
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)       
        
        for classe in self.classes:
            
            probabilityPerClass[classe] = self.logprob[classe]
            
            for word in words:
                if word in self.vector_of_words:
                    probabilityPerClass[classe]+= self.loglikelihood[word,classe]
        
        return max(probabilityPerClass, key=probabilityPerClass.get)
    
    def testNBClassifier(self):
        #path: caminho para o dataset test
        #buscar na pasta teste todos os arquivos e posteriormente chama predict()
        
        folders=os.listdir('./dataset/test')
        y_test={}
        y_predict={}
        
        
        for classe in folders:
            number_of_docs = len(os.listdir('./dataset/test/'+classe))
            y_test[classe]=number_of_docs            
            y_predict[classe]=0
        
        
        t_inicial = time.time()  
        
        for classe in folders:  
            files=os.listdir('./dataset/test/'+classe)
            for doc in files: 
                predict_class = self.predict('./dataset/test/'+classe+'/'+doc)
                y_predict[predict_class]+=1 
                
        t_final = time.time()
        
        tempo_total = t_final - t_inicial
        print("elapsed time: " +str(tempo_total))
          
            
        
    def getTrainingData(self):
        folders=os.listdir('./dataset/train')
        for classe in folders:
            number_of_docs = len(os.listdir('./dataset/train/'+classe))
            self.classes[classe]=number_of_docs        
          
    
    def extractVocabulary(self):
        bigtext=''
        folders=os.listdir('./dataset/train')
        for classe in folders:            
            files=os.listdir('./dataset/train/'+classe)
            for doc in files:
                bigtext += self.read_file("./dataset/train/"+classe+"/"+doc)
        normalizer = TextNormalizer()
        bigtext = normalizer.normalizing(bigtext)
        sentences = nltk.sent_tokenize(bigtext)        
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            words= [w for w in words if w.isalpha()] # remove all tokens that are not alphabetic
            #word_list = [w for w in words if not w in nltk.corpus.stopwords.words('english')]
           
        return set(words)
            
    
    def read_file(self,filename):
        with open (filename, 'r') as file:
            text = file.read()
        return text    
    
    def save(self, model):           
        f = open('NBclassifier','w')
        f.write(str(classifier))  
        f.close()
        
    def importModel(self, filename):
        model= self.read_file(filename)
        model= eval(model)
        self.classes = model[0]
        self.logprob = model[1]
        self.vector_of_words = model[2]
        self.loglikelihood = model[3]
    

In [13]:
if __name__ == "__main__":
    model = Model()  
    classifier = model.trainNBClassifier()
    model.save(classifier)   

{'neg': 3, 'pos': 2}
{'neg': -0.5108256237659907, 'pos': -0.916290731874155}
{'few', 'energy', 'boring', 'the', 'entirely', 'fun', 'summer', 'surprises', 'very', 'laughs', 'predictable', 'powerful', 'plain', 'most', 'film', 'just', 'no', 'of', 'lacks', 'and'}
{('few', 'neg'): 1.0294117647058822, ('energy', 'neg'): 1.0294117647058822, ('boring', 'neg'): 1.0294117647058822, ('the', 'neg'): 0.029411764705882353, ('entirely', 'neg'): 1.0294117647058822, ('fun', 'neg'): 0.029411764705882353, ('summer', 'neg'): 0.029411764705882353, ('surprises', 'neg'): 1.0294117647058822, ('very', 'neg'): 1.0294117647058822, ('laughs', 'neg'): 1.0294117647058822, ('predictable', 'neg'): 1.0294117647058822, ('powerful', 'neg'): 0.029411764705882353, ('plain', 'neg'): 1.0294117647058822, ('most', 'neg'): 0.029411764705882353, ('film', 'neg'): 0.029411764705882353, ('just', 'neg'): 1.0294117647058822, ('no', 'neg'): 1.0294117647058822, ('of', 'neg'): 0.029411764705882353, ('lacks', 'neg'): 1.0294117647058822,

In [14]:
new_model = Model()
model.importModel('NBclassifier')
model.testNBClassifier()


elapsed time: 0.00026726722717285156
